# Data Preparation:

##### Steps: 
- Extrating Data.
- Changing Time format from Epoch Unix to Human Readable time.
- Resampling Data.
- Saving to CSV File.

In [2]:
import pandas as pd
import numpy as np
import prophet as pt
import datetime as dt
import os
import matplotlib.pyplot as plt
from prophet.plot import add_changepoints_to_plot

## 1. Extracting Data and changing time format:

We now trying to clean and prepare the data, first starting by extracting data from the original files and add them up to be in one csv file that can be flexible for future use and also to reduce the size of the data, current is arround 15 GB which is a huge size.

In [3]:
#listing all files in the directory 
files  = os.listdir()
# Defining the names of the columns of the data:
columns = ["grid_square", "time", "cc", "sms_in", "sms_out", "call_in", "call_out", "internet"]

In [3]:
def convert(t):
    """
    Function to convert the time steops to a standard form add 
    subtracting the hour to be in the GMT time.
    """
    return dt.datetime.fromtimestamp(t / 1e3) - dt.timedelta(hours = 1)

In [4]:
files

['.ipynb_checkpoints',
 'new-final_data_preperation.ipynb',
 'sms-call-internet-mi-2013-11-01.txt',
 'sms-call-internet-mi-2013-11-02.txt',
 'sms-call-internet-mi-2013-11-03.txt',
 'sms-call-internet-mi-2013-11-04.txt',
 'sms-call-internet-mi-2013-11-05.txt',
 'sms-call-internet-mi-2013-11-06.txt',
 'sms-call-internet-mi-2013-11-07.txt',
 'sms-call-internet-mi-2013-11-08.txt',
 'sms-call-internet-mi-2013-11-09.txt',
 'sms-call-internet-mi-2013-11-10.txt',
 'sms-call-internet-mi-2013-11-11.txt',
 'sms-call-internet-mi-2013-11-12.txt',
 'sms-call-internet-mi-2013-11-13.txt',
 'sms-call-internet-mi-2013-11-14.txt',
 'sms-call-internet-mi-2013-11-15.txt',
 'sms-call-internet-mi-2013-11-16.txt',
 'sms-call-internet-mi-2013-11-17.txt',
 'sms-call-internet-mi-2013-11-18.txt',
 'sms-call-internet-mi-2013-11-19.txt',
 'sms-call-internet-mi-2013-11-20.txt',
 'sms-call-internet-mi-2013-11-21.txt',
 'sms-call-internet-mi-2013-11-22.txt',
 'sms-call-internet-mi-2013-11-23.txt',
 'sms-call-internet-

As we can see here, not all files are our data files, so we have to start from the third index from files to get the correct data files, which is 62 files every file represets a measurment for a day, then we hae to iterate through all of the data add them up, and also taking into consideration that the findings that we have found about roaming CDRs and different tyes of traffic so the next step includes:
- Reading the files which was tap separated values
- Rename the columns to be the desired ones.
- Converting time format for all time.
- Convert time column to a datetime formate to be able to work with as a time series well.
- Excluding all Roaming Data
- Excluding all other CDRs except internet CDRs.
- Add all files to gether
- Resampling Data.
- Output them to a csv file.

In [6]:
#Iterate over all files
full_data = pd.DataFrame()
for f in files[2:]:
    #get day number
    day = f[-9:-4]
    #Read the file
    df = pd.read_csv(f,sep="\t", header=None, names= columns)    
    #convert time column to a readable time
    df["time"] = df["time"].apply(convert)
    df["time"] = pd.to_datetime(df["time"])
    #select italy dataframe
    df = df[df["cc"] == 39]
    #grouping the data (each time step we have a value for each grid)
    df = df.groupby(['time','grid_square']).sum()
    #define a new aggregated column
    df["internet_cdr"] = df.loc[:, ["internet"]]
    #drop the unwanted columns
    df.drop(columns = ["cc","sms_in", "sms_out", "call_in", "call_out", "internet"], inplace = True)
    #Concate df with full data
    full_data = pd.concat([full_data, df])
    #save the data to a csv
    #Check point
    print('===========> Day {} is done!'.format(day))
    print('===========> Full data rows: {} rows'.format(len(full_data)))

full_data.to_csv('full_grid.csv')
print('===========> Full data size: {} Mb'.format(os.path.getsize('full_grid.csv')/10**6))

===========> Day 11-01 is done!
===========> Full data rows: 1439981 rows
===========> Day 11-02 is done!
===========> Full data rows: 2879964 rows
===========> Day 11-03 is done!
===========> Full data rows: 4319921 rows
===========> Day 11-04 is done!
===========> Full data rows: 5759893 rows
===========> Day 11-05 is done!
===========> Full data rows: 7199869 rows
===========> Day 11-06 is done!
===========> Full data rows: 8639843 rows
===========> Day 11-07 is done!
===========> Full data rows: 10079821 rows
===========> Day 11-08 is done!
===========> Full data rows: 11519793 rows
===========> Day 11-09 is done!
===========> Full data rows: 12959768 rows
===========> Day 11-10 is done!
===========> Full data rows: 14399750 rows
===========> Day 11-11 is done!
===========> Full data rows: 15839734 rows
===========> Day 11-12 is done!
===========> Full data rows: 17279706 rows
===========> Day 11-13 is done!
===========> Full data rows: 18719680 rows
===========> Day 11-14 is done!

as we can see the number of rows on the data = 89243075, and the file size is arround 4 GB only, but our desired number of rows have to be = `144(samples perday)*10000(grids)*62(days) = 89 280 000` not 89243075 that means that there are some measurements for some grids that wasn't taken or recorded in the data but to make our interval consistant we have to resample the data on time intervl of 10 mins to get the actual data, that will increase the Null values in the data but we can deal with it later.

## 2. Resampling Data every 10 mins:

In [7]:
# Reading the file that we have saved.
df = pd.read_csv("full_grid.csv")

In [8]:
# Parse Datafrmae
df = df.reset_index()
df["time"] = pd.to_datetime(df["time"])
df = df.set_index("time")
df.drop(columns = ["index"], inplace = True)
df = df.groupby("grid_square")
df_groups = list(df.groups.keys())

First of all we need to know all the grids that have missing records, that will help us to reample those grids only instead of all grids to same computation power and time in addtion to knowing how many records were missed from the data.

In [28]:
missing_grids = {}
for grid in df_groups:
    if df.get_group(grid).shape[0] != 8928:
        missing_grids[grid] = 8928 - df.get_group(grid).shape[0]

In [29]:
len(missing_grids)

94

We Have 94 grids that have missing records, now let's show them:

In [30]:
missing_grids

{94: 245,
 95: 245,
 96: 245,
 97: 245,
 98: 16,
 99: 245,
 100: 245,
 195: 245,
 196: 245,
 197: 245,
 198: 245,
 199: 245,
 200: 224,
 296: 245,
 297: 245,
 298: 245,
 299: 224,
 300: 50,
 397: 217,
 398: 48,
 399: 46,
 400: 47,
 497: 48,
 498: 46,
 499: 137,
 500: 137,
 598: 147,
 599: 137,
 600: 137,
 698: 147,
 699: 147,
 700: 147,
 799: 147,
 800: 147,
 2049: 45,
 2149: 45,
 2250: 37,
 2350: 42,
 2551: 42,
 2651: 42,
 4374: 1172,
 4471: 780,
 4472: 279,
 4473: 524,
 4474: 1151,
 4570: 501,
 4571: 348,
 4572: 173,
 4573: 418,
 4574: 826,
 4575: 765,
 4671: 400,
 4672: 461,
 4673: 418,
 4674: 482,
 4675: 1443,
 4771: 319,
 4772: 506,
 4773: 807,
 4774: 411,
 4775: 508,
 4875: 140,
 4975: 305,
 4993: 9,
 5076: 590,
 5092: 9,
 5093: 9,
 5192: 9,
 5193: 9,
 5239: 6108,
 5339: 5190,
 5340: 1210,
 5439: 2100,
 5440: 2100,
 6189: 10,
 6190: 13,
 7623: 26,
 7625: 26,
 7722: 61,
 7727: 175,
 7823: 1,
 7824: 5,
 7825: 1,
 7826: 33,
 7925: 2,
 9247: 1,
 9248: 1,
 9347: 5,
 9348: 1,
 9349: 1,

In [33]:
np.sum(list(missing_grids.values()))

36925

We have a 36925 records that was missing from the data, now we have to resemple the data.

In [34]:
final_df = pd.DataFrame()
for grid in df_groups:
    df_g = df.get_group(grid)
    if grid in missing_grids:
        # ===== Resampling by aggregating if there is any records between 10 mins (not in our data)========
        df_g1 = df_g.resample("10min", origin="2013-11-1 00:00:00").sum()
        # Replacing zeros with nan values in internet column
        df_g1[["internet_cdr"]] = df_g1[["internet_cdr"]].replace({0: np.nan})
        # Replacing grid_square with grid num in grid square columns
        df_g1[["grid_square"]] = df_g1[["grid_square"]].replace({0: grid})
        # Concatination
        final_df = pd.concat([final_df,df_g1])
    else:
        final_df = pd.concat([final_df,df_g])
    print("============= grid {} resampled =====".format(grid))

============= grid 1 resampled =====
============= grid 2 resampled =====
============= grid 3 resampled =====
============= grid 4 resampled =====
============= grid 5 resampled =====
============= grid 6 resampled =====
============= grid 7 resampled =====
============= grid 8 resampled =====
============= grid 9 resampled =====
============= grid 10 resampled =====
============= grid 11 resampled =====
============= grid 12 resampled =====
============= grid 13 resampled =====
============= grid 14 resampled =====
============= grid 15 resampled =====
============= grid 16 resampled =====
============= grid 17 resampled =====
============= grid 18 resampled =====
============= grid 19 resampled =====
============= grid 20 resampled =====
============= grid 21 resampled =====
============= grid 22 resampled =====
============= grid 23 resampled =====
============= grid 24 resampled =====
============= grid 25 resampled =====
============= grid 26 resampled =====
============= grid 27

============= grid 219 resampled =====
============= grid 220 resampled =====
============= grid 221 resampled =====
============= grid 222 resampled =====
============= grid 223 resampled =====
============= grid 224 resampled =====
============= grid 225 resampled =====
============= grid 226 resampled =====
============= grid 227 resampled =====
============= grid 228 resampled =====
============= grid 229 resampled =====
============= grid 230 resampled =====
============= grid 231 resampled =====
============= grid 232 resampled =====
============= grid 233 resampled =====
============= grid 234 resampled =====
============= grid 235 resampled =====
============= grid 236 resampled =====
============= grid 237 resampled =====
============= grid 238 resampled =====
============= grid 239 resampled =====
============= grid 240 resampled =====
============= grid 241 resampled =====
============= grid 242 resampled =====
============= grid 243 resampled =====
============= grid 244 re

============= grid 435 resampled =====
============= grid 436 resampled =====
============= grid 437 resampled =====
============= grid 438 resampled =====
============= grid 439 resampled =====
============= grid 440 resampled =====
============= grid 441 resampled =====
============= grid 442 resampled =====
============= grid 443 resampled =====
============= grid 444 resampled =====
============= grid 445 resampled =====
============= grid 446 resampled =====
============= grid 447 resampled =====
============= grid 448 resampled =====
============= grid 449 resampled =====
============= grid 450 resampled =====
============= grid 451 resampled =====
============= grid 452 resampled =====
============= grid 453 resampled =====
============= grid 454 resampled =====
============= grid 455 resampled =====
============= grid 456 resampled =====
============= grid 457 resampled =====
============= grid 458 resampled =====
============= grid 459 resampled =====
============= grid 460 re

============= grid 646 resampled =====
============= grid 647 resampled =====
============= grid 648 resampled =====
============= grid 649 resampled =====
============= grid 650 resampled =====
============= grid 651 resampled =====
============= grid 652 resampled =====
============= grid 653 resampled =====
============= grid 654 resampled =====
============= grid 655 resampled =====
============= grid 656 resampled =====
============= grid 657 resampled =====
============= grid 658 resampled =====
============= grid 659 resampled =====
============= grid 660 resampled =====
============= grid 661 resampled =====
============= grid 662 resampled =====
============= grid 663 resampled =====
============= grid 664 resampled =====
============= grid 665 resampled =====
============= grid 666 resampled =====
============= grid 667 resampled =====
============= grid 668 resampled =====
============= grid 669 resampled =====
============= grid 670 resampled =====
============= grid 671 re

============= grid 859 resampled =====
============= grid 860 resampled =====
============= grid 861 resampled =====
============= grid 862 resampled =====
============= grid 863 resampled =====
============= grid 864 resampled =====
============= grid 865 resampled =====
============= grid 866 resampled =====
============= grid 867 resampled =====
============= grid 868 resampled =====
============= grid 869 resampled =====
============= grid 870 resampled =====
============= grid 871 resampled =====
============= grid 872 resampled =====
============= grid 873 resampled =====
============= grid 874 resampled =====
============= grid 875 resampled =====
============= grid 876 resampled =====
============= grid 877 resampled =====
============= grid 878 resampled =====
============= grid 879 resampled =====
============= grid 880 resampled =====
============= grid 881 resampled =====
============= grid 882 resampled =====
============= grid 883 resampled =====
============= grid 884 re

...

============= grid 9068 resampled =====
============= grid 9069 resampled =====
============= grid 9070 resampled =====
============= grid 9071 resampled =====
============= grid 9072 resampled =====
============= grid 9073 resampled =====
============= grid 9074 resampled =====
============= grid 9075 resampled =====
============= grid 9076 resampled =====
============= grid 9077 resampled =====
============= grid 9078 resampled =====
============= grid 9079 resampled =====
============= grid 9080 resampled =====
============= grid 9081 resampled =====
============= grid 9082 resampled =====
============= grid 9083 resampled =====
============= grid 9084 resampled =====
============= grid 9085 resampled =====
============= grid 9086 resampled =====
============= grid 9087 resampled =====
============= grid 9088 resampled =====
============= grid 9089 resampled =====
============= grid 9090 resampled =====
============= grid 9091 resampled =====
============= grid 9092 resampled =====


============= grid 9273 resampled =====
============= grid 9274 resampled =====
============= grid 9275 resampled =====
============= grid 9276 resampled =====
============= grid 9277 resampled =====
============= grid 9278 resampled =====
============= grid 9279 resampled =====
============= grid 9280 resampled =====
============= grid 9281 resampled =====
============= grid 9282 resampled =====
============= grid 9283 resampled =====
============= grid 9284 resampled =====
============= grid 9285 resampled =====
============= grid 9286 resampled =====
============= grid 9287 resampled =====
============= grid 9288 resampled =====
============= grid 9289 resampled =====
============= grid 9290 resampled =====
============= grid 9291 resampled =====
============= grid 9292 resampled =====
============= grid 9293 resampled =====
============= grid 9294 resampled =====
============= grid 9295 resampled =====
============= grid 9296 resampled =====
============= grid 9297 resampled =====


============= grid 9478 resampled =====
============= grid 9479 resampled =====
============= grid 9480 resampled =====
============= grid 9481 resampled =====
============= grid 9482 resampled =====
============= grid 9483 resampled =====
============= grid 9484 resampled =====
============= grid 9485 resampled =====
============= grid 9486 resampled =====
============= grid 9487 resampled =====
============= grid 9488 resampled =====
============= grid 9489 resampled =====
============= grid 9490 resampled =====
============= grid 9491 resampled =====
============= grid 9492 resampled =====
============= grid 9493 resampled =====
============= grid 9494 resampled =====
============= grid 9495 resampled =====
============= grid 9496 resampled =====
============= grid 9497 resampled =====
============= grid 9498 resampled =====
============= grid 9499 resampled =====
============= grid 9500 resampled =====
============= grid 9501 resampled =====
============= grid 9502 resampled =====


============= grid 9683 resampled =====
============= grid 9684 resampled =====
============= grid 9685 resampled =====
============= grid 9686 resampled =====
============= grid 9687 resampled =====
============= grid 9688 resampled =====
============= grid 9689 resampled =====
============= grid 9690 resampled =====
============= grid 9691 resampled =====
============= grid 9692 resampled =====
============= grid 9693 resampled =====
============= grid 9694 resampled =====
============= grid 9695 resampled =====
============= grid 9696 resampled =====
============= grid 9697 resampled =====
============= grid 9698 resampled =====
============= grid 9699 resampled =====
============= grid 9700 resampled =====
============= grid 9701 resampled =====
============= grid 9702 resampled =====
============= grid 9703 resampled =====
============= grid 9704 resampled =====
============= grid 9705 resampled =====
============= grid 9706 resampled =====
============= grid 9707 resampled =====


============= grid 9888 resampled =====
============= grid 9889 resampled =====
============= grid 9890 resampled =====
============= grid 9891 resampled =====
============= grid 9892 resampled =====
============= grid 9893 resampled =====
============= grid 9894 resampled =====
============= grid 9895 resampled =====
============= grid 9896 resampled =====
============= grid 9897 resampled =====
============= grid 9898 resampled =====
============= grid 9899 resampled =====
============= grid 9900 resampled =====
============= grid 9901 resampled =====
============= grid 9902 resampled =====
============= grid 9903 resampled =====
============= grid 9904 resampled =====
============= grid 9905 resampled =====
============= grid 9906 resampled =====
============= grid 9907 resampled =====
============= grid 9908 resampled =====
============= grid 9909 resampled =====
============= grid 9910 resampled =====
============= grid 9911 resampled =====
============= grid 9912 resampled =====


### Validating the resampling :

In [35]:
final_df.reset_index()["time"].value_counts().sum()

89270147

After this resampling we found that there is some some values that wasn't resampled well, so wee need to try another method to resmaple the data.

In [36]:
final_df.isna().sum()

grid_square          0
internet_cdr    142686
dtype: int64

In [37]:
missing_grids = {}
lm = final_df.groupby("grid_square")

In [38]:
for grid in df_groups:
    if lm.get_group(grid).shape[0] != 8928:
        missing_grids[grid] = 8928 - lm.get_group(grid).shape[0]

Getting the cells thas has some absent measurments

In [39]:
missing_grids

{4675: 3, 4773: 2, 5239: 4953, 5339: 4891, 5439: 2, 5440: 2}

Now as we can see the number of grids that have missing data was reduced to only 6 grids and the missing records to only 9900 records.

In [40]:
#saving grids with missed values in a list
miss_grids = list(missing_grids.keys())

In [41]:
#Taking a complete cell as a reference to get all possible dates 
re_grid = set(list(lm.get_group(39).resample("10min").asfreq().index))

In [62]:
ff= pd.DataFrame()
for grid in df_groups:
    gg = lm.get_group(grid)
    if grid in miss_grids:
        # Getting the dates that were in the data
        dts = set(list(gg.resample("10min").asfreq().index))
        # Get the missed dates by subtracting the complete time with this list of times.
        missed = list(re_grid - dts)
        final_missed = []
        # Format the date to the right form
        for date in missed:
            #Make sure that the time was parsed the right way before resampling.
            d = pd.to_datetime(date).strftime("%Y-%m-%d %H:%M:%S")
            d = pd.to_datetime(d,format ="%Y-%m-%d %H:%M:%S")
            final_missed.append(d)
        # adding the Nan measurement to it's corresponding position
        for date2 in final_missed:
            # Making the row data maually
            row = pd.DataFrame({"grid_square": grid, "internet_cdr": np.nan}, index = [date2])
            # Cancat the old data with the resampled row
            gg = pd.concat([gg[:date2], row, gg[date2:]])
        ff = pd.concat([ff,gg])
    else:
        ff = pd.concat([ff,gg])
    print("============= grid {} resampled =====".format(grid))

============= grid 1 resampled =====
============= grid 2 resampled =====
============= grid 3 resampled =====
============= grid 4 resampled =====
============= grid 5 resampled =====
============= grid 6 resampled =====
============= grid 7 resampled =====
============= grid 8 resampled =====
============= grid 9 resampled =====
============= grid 10 resampled =====
============= grid 11 resampled =====
============= grid 12 resampled =====
============= grid 13 resampled =====
============= grid 14 resampled =====
============= grid 15 resampled =====
============= grid 16 resampled =====
============= grid 17 resampled =====
============= grid 18 resampled =====
============= grid 19 resampled =====
============= grid 20 resampled =====
============= grid 21 resampled =====
============= grid 22 resampled =====
============= grid 23 resampled =====
============= grid 24 resampled =====
============= grid 25 resampled =====
============= grid 26 resampled =====
============= grid 27

============= grid 220 resampled =====
============= grid 221 resampled =====
============= grid 222 resampled =====
============= grid 223 resampled =====
============= grid 224 resampled =====
============= grid 225 resampled =====
============= grid 226 resampled =====
============= grid 227 resampled =====
============= grid 228 resampled =====
============= grid 229 resampled =====
============= grid 230 resampled =====
============= grid 231 resampled =====
============= grid 232 resampled =====
============= grid 233 resampled =====
============= grid 234 resampled =====
============= grid 235 resampled =====
============= grid 236 resampled =====
============= grid 237 resampled =====
============= grid 238 resampled =====
============= grid 239 resampled =====
============= grid 240 resampled =====
============= grid 241 resampled =====
============= grid 242 resampled =====
============= grid 243 resampled =====
============= grid 244 resampled =====
============= grid 245 re

============= grid 432 resampled =====
============= grid 433 resampled =====
============= grid 434 resampled =====
============= grid 435 resampled =====
============= grid 436 resampled =====
============= grid 437 resampled =====
============= grid 438 resampled =====
============= grid 439 resampled =====
============= grid 440 resampled =====
============= grid 441 resampled =====
============= grid 442 resampled =====
============= grid 443 resampled =====
============= grid 444 resampled =====
============= grid 445 resampled =====
============= grid 446 resampled =====
============= grid 447 resampled =====
============= grid 448 resampled =====
============= grid 449 resampled =====
============= grid 450 resampled =====
============= grid 451 resampled =====
============= grid 452 resampled =====
============= grid 453 resampled =====
============= grid 454 resampled =====
============= grid 455 resampled =====
============= grid 456 resampled =====
============= grid 457 re

============= grid 643 resampled =====
============= grid 644 resampled =====
============= grid 645 resampled =====
============= grid 646 resampled =====
============= grid 647 resampled =====
============= grid 648 resampled =====
============= grid 649 resampled =====
============= grid 650 resampled =====
============= grid 651 resampled =====
============= grid 652 resampled =====
============= grid 653 resampled =====
============= grid 654 resampled =====
============= grid 655 resampled =====
============= grid 656 resampled =====
============= grid 657 resampled =====
============= grid 658 resampled =====
============= grid 659 resampled =====
============= grid 660 resampled =====
============= grid 661 resampled =====
============= grid 662 resampled =====
============= grid 663 resampled =====
============= grid 664 resampled =====
============= grid 665 resampled =====
============= grid 666 resampled =====
============= grid 667 resampled =====
============= grid 668 re

============= grid 856 resampled =====
============= grid 857 resampled =====
============= grid 858 resampled =====
============= grid 859 resampled =====
============= grid 860 resampled =====
============= grid 861 resampled =====
============= grid 862 resampled =====
============= grid 863 resampled =====
============= grid 864 resampled =====
============= grid 865 resampled =====
============= grid 866 resampled =====
============= grid 867 resampled =====
============= grid 868 resampled =====
============= grid 869 resampled =====
============= grid 870 resampled =====
============= grid 871 resampled =====
============= grid 872 resampled =====
============= grid 873 resampled =====
============= grid 874 resampled =====
============= grid 875 resampled =====
============= grid 876 resampled =====
============= grid 877 resampled =====
============= grid 878 resampled =====
============= grid 879 resampled =====
============= grid 880 resampled =====
============= grid 881 re

...

============= grid 9066 resampled =====
============= grid 9067 resampled =====
============= grid 9068 resampled =====
============= grid 9069 resampled =====
============= grid 9070 resampled =====
============= grid 9071 resampled =====
============= grid 9072 resampled =====
============= grid 9073 resampled =====
============= grid 9074 resampled =====
============= grid 9075 resampled =====
============= grid 9076 resampled =====
============= grid 9077 resampled =====
============= grid 9078 resampled =====
============= grid 9079 resampled =====
============= grid 9080 resampled =====
============= grid 9081 resampled =====
============= grid 9082 resampled =====
============= grid 9083 resampled =====
============= grid 9084 resampled =====
============= grid 9085 resampled =====
============= grid 9086 resampled =====
============= grid 9087 resampled =====
============= grid 9088 resampled =====
============= grid 9089 resampled =====
============= grid 9090 resampled =====


============= grid 9271 resampled =====
============= grid 9272 resampled =====
============= grid 9273 resampled =====
============= grid 9274 resampled =====
============= grid 9275 resampled =====
============= grid 9276 resampled =====
============= grid 9277 resampled =====
============= grid 9278 resampled =====
============= grid 9279 resampled =====
============= grid 9280 resampled =====
============= grid 9281 resampled =====
============= grid 9282 resampled =====
============= grid 9283 resampled =====
============= grid 9284 resampled =====
============= grid 9285 resampled =====
============= grid 9286 resampled =====
============= grid 9287 resampled =====
============= grid 9288 resampled =====
============= grid 9289 resampled =====
============= grid 9290 resampled =====
============= grid 9291 resampled =====
============= grid 9292 resampled =====
============= grid 9293 resampled =====
============= grid 9294 resampled =====
============= grid 9295 resampled =====


============= grid 9476 resampled =====
============= grid 9477 resampled =====
============= grid 9478 resampled =====
============= grid 9479 resampled =====
============= grid 9480 resampled =====
============= grid 9481 resampled =====
============= grid 9482 resampled =====
============= grid 9483 resampled =====
============= grid 9484 resampled =====
============= grid 9485 resampled =====
============= grid 9486 resampled =====
============= grid 9487 resampled =====
============= grid 9488 resampled =====
============= grid 9489 resampled =====
============= grid 9490 resampled =====
============= grid 9491 resampled =====
============= grid 9492 resampled =====
============= grid 9493 resampled =====
============= grid 9494 resampled =====
============= grid 9495 resampled =====
============= grid 9496 resampled =====
============= grid 9497 resampled =====
============= grid 9498 resampled =====
============= grid 9499 resampled =====
============= grid 9500 resampled =====


============= grid 9681 resampled =====
============= grid 9682 resampled =====
============= grid 9683 resampled =====
============= grid 9684 resampled =====
============= grid 9685 resampled =====
============= grid 9686 resampled =====
============= grid 9687 resampled =====
============= grid 9688 resampled =====
============= grid 9689 resampled =====
============= grid 9690 resampled =====
============= grid 9691 resampled =====
============= grid 9692 resampled =====
============= grid 9693 resampled =====
============= grid 9694 resampled =====
============= grid 9695 resampled =====
============= grid 9696 resampled =====
============= grid 9697 resampled =====
============= grid 9698 resampled =====
============= grid 9699 resampled =====
============= grid 9700 resampled =====
============= grid 9701 resampled =====
============= grid 9702 resampled =====
============= grid 9703 resampled =====
============= grid 9704 resampled =====
============= grid 9705 resampled =====


============= grid 9886 resampled =====
============= grid 9887 resampled =====
============= grid 9888 resampled =====
============= grid 9889 resampled =====
============= grid 9890 resampled =====
============= grid 9891 resampled =====
============= grid 9892 resampled =====
============= grid 9893 resampled =====
============= grid 9894 resampled =====
============= grid 9895 resampled =====
============= grid 9896 resampled =====
============= grid 9897 resampled =====
============= grid 9898 resampled =====
============= grid 9899 resampled =====
============= grid 9900 resampled =====
============= grid 9901 resampled =====
============= grid 9902 resampled =====
============= grid 9903 resampled =====
============= grid 9904 resampled =====
============= grid 9905 resampled =====
============= grid 9906 resampled =====
============= grid 9907 resampled =====
============= grid 9908 resampled =====
============= grid 9909 resampled =====
============= grid 9910 resampled =====


In [63]:
ff.shape

(89280000, 2)

finally the data have been resampled well!

In [64]:
ff.isna().sum()

grid_square          0
internet_cdr    152539
dtype: int64

there are 152539 missing measurements!

## 3. Saving the results to csv:

In [66]:
ff = ff.reset_index().rename(columns = {"index": "time"})

In [89]:
ff.to_csv("resampled_full_grid.csv", index = False)

In [47]:
ff

,time,grid_square,internet_cdr
0,2013-11-01 00:00:00,1,11.028366
1,2013-11-01 00:10:00,1,11.100963
2,2013-11-01 00:20:00,1,10.892771
3,2013-11-01 00:30:00,1,8.622425
4,2013-11-01 00:40:00,1,8.009927
...,...,...,...
89279995,2014-01-01 23:10:00,10000,13.598430
89279996,2014-01-01 23:20:00,10000,11.312810
89279997,2014-01-01 23:30:00,10000,12.594609
89279998,2014-01-01 23:40:00,10000,9.885762
